# Agent: Recovery

이 튜토리얼을 통해 다음을 배울 수 있다:

- AI 에이전트에서 복구 메커니즘의 필요성을 이해한다.
- 재시도 로직과 폴백 전략을 구현한다.
- 지수 백오프와 서킷 브레이커 패턴을 적용한다.
- 안정적이고 복원력 있는 에이전트를 설계한다.

## 1. 환경 설정

### 1.1 필요한 라이브러리 설치

In [1]:
#%pip install openai pydantic instructor python-dotenv 
%pip install -q tenacity

Note: you may need to restart the kernel to use updated packages.


### 1.2 환경 설정

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

MODEL = "gpt-4o-mini"

### 1.3 필요한 라이브러리 가져오기

In [3]:
from typing import Optional
from openai import OpenAI
from pydantic import BaseModel, Field
import instructor
import time

## 2. 기본 복구 패턴

### 2.1 단순 폴백 전략
데이터가 없거나 오류가 발생하면 대체 응답을 제공한다.

In [5]:
client = instructor.patch(OpenAI())

class UserInfo(BaseModel):
    """사용자 정보 모델이다."""
    name: str
    email: str
    age: Optional[int] = None

def extract_user_info_with_fallback(text: str) -> str:
    """사용자 정보를 추출하고 실패 시 폴백한다."""
    try:
        user = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": "텍스트에서 사용자 정보를 추출한다."},
                {"role": "user", "content": text}
            ],
            response_model=UserInfo,
        )
        # age 정보가 있을 경우 age를 포함한 응답을 반환한다
        if user.age is not None:
            return f"{user.name}님의 나이는 {user.age}세다."
        # age 정보가 없을 경우 (None일 경우) 폴백 응답을 반환한다
        else:
            return f"{user.name}님의 이메일은 {user.email}다. (나이 정보 없음)"
    except Exception as e:
        # LLM 호출 자체에서 에러가 발생한 경우에 대한 폴백이다
        return f"정보 추출에 실패했습니다: {e}"

# 사용 예시
result = extract_user_info_with_fallback(
    "제 이름은 김철수이고 이메일은 chulsoo@example.com입니다."
)
print(result)

김철수님의 이메일은 chulsoo@example.com다. (나이 정보 없음)


### 2.2 재시도 로직
일시적 오류는 다시 시도하여 해결한다.

In [6]:
def call_with_retry(prompt: str, max_retries: int = 3) -> str:
    """재시도 로직이 포함된 호출 함수다."""
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[{"role": "user", "content": prompt}]
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"시도 {attempt + 1} 실패: {str(e)}")
            # 마지막 시도가 아니면 잠시 대기한다
            if attempt < max_retries - 1:
                time.sleep(1)
    
    return "최대 재시도 횟수를 초과했습니다."

# 사용 예시
# 정상적인 경우, 첫 시도에 성공한다
result = call_with_retry("AI란 무엇인가?")
print(result)

AI(인공지능)는 인간의 지능을 모방하거나 시뮬레이션하는 컴퓨터 시스템이나 소프트웨어입니다. AI는 다양한 기술과 알고리즘을 사용하여 패턴 인식, 학습, 문제 해결, 언어 이해, 자율적 의사결정 등을 수행할 수 있습니다. 

AI는 크게 두 가지 유형으로 나눌 수 있습니다:

1. **좁은 인공지능(Narrow AI)**: 특정 작업이나 문제를 해결하는 데 특화된 AI입니다. 예를 들어, 음성 인식 소프트웨어, 추천 시스템, 자율주행 자동차의 인식 시스템 등이 이에 해당합니다.

2. **강한 인공지능(General AI)**: 인간과 같은 수준의 지능을 가지며, 다양한 문제를 해결할 수 있는 AI를 말합니다. 현재까지 강한 인공지능은 실현되지 않았으며, 연구가 진행 중입니다.

AI는 머신러닝(기계 학습), 딥러닝(심층 학습), 자연어 처리(NLP), 컴퓨터 비전 등과 같은 하위 분야를 포함합니다. 이러한 기술들은 AI가 데이터를 분석하고 학습하여 자신의 성능을 향상시키는 데 도움을 줍니다. AI는 의료, 금융, 제조, 엔터테인먼트 등 다양한 산업에서 활용되고 있습니다.


### 2.3 데코레이터 패턴
재사용 가능한 복구 로직을 데코레이터로 구현한다.

In [7]:
from functools import wraps

def with_fallback(fallback_value):
    """폴백 값을 제공하는 데코레이터다."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except Exception as e:
                print(f"함수 '{func.__name__}' 실행 중 오류 발생: {e}. 폴백 값을 반환합니다.")
                return fallback_value
        return wrapper
    return decorator

@with_fallback("기본 응답을 제공합니다.")
def risky_function(prompt: str) -> str:
    """오류가 발생할 수 있는 함수다."""
    # 오류를 인위적으로 발생시켜본다
    # raise ValueError("인위적인 오류 발생")
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# 사용 예시
result = risky_function("테스트 질문")
print(result)

안녕하세요! 어떤 질문이든지 해주시면 최선을 다해 답변해드리겠습니다. 무엇을 도와드릴까요?


## 3. 고급 복구 패턴

### 3.1 지수 백오프
재시도 간격을 지수적으로 늘려서 서버 부하를 줄인다.

In [8]:
def exponential_backoff_retry(prompt: str, max_retries: int = 5) -> str:
    """지수 백오프를 사용한 재시도 함수다."""
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[{"role": "user", "content": prompt}],
                timeout=10
            )
            return response.choices[0].message.content
        except Exception as e:
            wait_time = 2 ** attempt
            print(f"시도 {attempt + 1} 실패: {e}. {wait_time}초 대기 후 재시도합니다...")
            time.sleep(wait_time)
    
    return "모든 재시도가 실패했습니다."

# 사용 예시
# 정상적인 경우, 1차 시도에 성공한다.
result = exponential_backoff_retry("복잡한 질문에 답해주세요")
print(result)

물론입니다! 복잡한 질문도 좋고, 어떤 주제든지 상관없습니다. 질문 해주시면 최선을 다해 답변해 드리겠습니다. 어떤 내용이 궁금하신가요?


### 3.2 Tenacity 라이브러리 활용
전문 재시도 라이브러리를 사용하여 더 강력한 복구를 구현한다.

In [9]:
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from openai import RateLimitError, APITimeoutError

@retry(
    stop=stop_after_attempt(3), # 최대 3번 시도한다
    wait=wait_exponential(multiplier=1, min=1, max=10), # 대기 시간을 지수적으로 늘린다
    retry=retry_if_exception_type((RateLimitError, APITimeoutError)) # 특정 예외 발생 시 재시도한다
)
def robust_llm_call(prompt: str) -> str:
    """Tenacity를 활용한 강력한 LLM 호출이다."""
    print("LLM 호출 시도...")
    # 오류를 인위적으로 발생시키려면 아래 주석을 해제한다
    # raise RateLimitError("Rate limit exceeded", response=None, body=None)
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# 사용 예시
try:
    result = robust_llm_call("안정적인 호출 테스트")
    print(result)
except Exception as e:
    print(f"최종 실패: {e}")

LLM 호출 시도...
안정적인 호출 테스트는 소프트웨어 개발 및 시스템 운영에서 중요한 단계입니다. 이는 시스템이 예상대로 작동하는지 확인하고, 특정 조건 하에서도 안정성을 유지하는지를 평가하는 과정입니다. 다음은 안정적인 호출 테스트를 수행하는 방법에 대한 몇 가지 가이드라인입니다.

1. **목표 정의**:
   - 테스트의 목표를 명확히 정의합니다. 어떤 특정한 기능이나 API의 응답성을 검증할 것인지 명시합니다.

2. **테스트 환경 준비**:
   - 테스트를 수행할 환경을 설정합니다. 이는 프로덕션 환경과 유사하게 구성되어야 하며, 오류 발생 시 영향을 최소화할 수 있도록 독립적인 환경을 구축하는 것이 좋습니다.

3. **테스트 시나리오 작성**:
   - 다양한 시나리오를 기반으로 테스트 케이스를 작성합니다. 예를 들어, 정상적인 호출, 비정상적인 입력값, 동시 호출 시나리오 등을 포함할 수 있습니다.

4. **부하 테스트**:
   - 시스템의 부하를 증가시키는 테스트를 수행하여 성능과 안정성을 평가합니다. 실제 사용할 트래픽을 시뮬레이션하기 위해 여러 클라이언트의 동시 호출을 테스트합니다.

5. **모니터링 및 로깅**:
   - 시스템 호출 중 발생하는 모든 이벤트를 모니터링하고 로깅하여 문제를 추적할 수 있도록 합니다. 에러율, 응답 시간 등을 기록합니다.

6. **결과 분석**:
   - 테스트 후 결과를 분석하여 문제가 발생한 경우 원인을 규명합니다. 정상 동작 여부와 성능 지표를 평가합니다.

7. **피드백 및 개선**:
   - 테스트 결과를 바탕으로 시스템을 개선합니다. 문제점을 해결하고 필요한 경우 시스템 아키텍처나 코드에 대한 수정 작업을 진행합니다.

8. **반복 테스트**:
   - 수정 후 다시 테스트를 수행하여 문제가 해결되었는지 확인합니다. 안정성을 확보할 때까지 이 과정을 반복합니다.

이러한 단계를 통해 안정적인 호출 테스트를 수행하면 시스템의 신뢰성을 높이고 사용자에게 더 나은 경험을 제공할 수 있습

### 3.3 서킷 브레이커 패턴
연속적인 실패가 발생하면 일정 시간 동안 요청을 차단하여 시스템 부하를 줄인다.

In [10]:
from datetime import datetime, timedelta

class CircuitBreaker:
    """서킷 브레이커 패턴 구현 클래스다."""
    
    def __init__(self, failure_threshold: int = 5, timeout: int = 60):
        self.failure_threshold = failure_threshold
        self.timeout = timeout
        self.failure_count = 0
        self.last_failure_time = None
        self.state = "CLOSED" # 상태: CLOSED, OPEN, HALF_OPEN
    
    def call(self, func, *args, **kwargs):
        """서킷 브레이커를 통한 함수 호출이다."""
        
        if self.state == "OPEN":
            time_since_failure = (datetime.now() - self.last_failure_time).seconds
            
            if time_since_failure < self.timeout:
                return "서킷이 열려있습니다. 잠시 후 다시 시도하세요."
            else:
                self.state = "HALF_OPEN"
                print("서킷이 HALF_OPEN 상태로 변경되었습니다. 테스트 호출을 시도합니다.")
        
        try:
            result = func(*args, **kwargs)
            self.on_success()
            return result
        except Exception as e:
            self.on_failure()
            return f"요청 실패: {str(e)}"
    
    def on_success(self):
        """성공 시 상태를 초기화한다."""
        self.failure_count = 0
        if self.state == "HALF_OPEN":
            self.state = "CLOSED"
            print("테스트 호출 성공. 서킷을 닫습니다.")
    
    def on_failure(self):
        """실패 시 카운트를 증가시킨다."""
        self.failure_count += 1
        self.last_failure_time = datetime.now()
        
        if self.state == "HALF_OPEN" or self.failure_count >= self.failure_threshold:
            self.state = "OPEN"
            print(f"서킷 브레이커 작동! {self.timeout}초 동안 차단됩니다.")

# 사용 예시
circuit_breaker = CircuitBreaker(failure_threshold=3, timeout=10)

def failing_api_call():
    """실패를 시뮬레이션하는 함수다."""
    raise ValueError("API 응답 실패")

for i in range(5):
    print(f"시도 {i+1}:")
    result = circuit_breaker.call(failing_api_call)
    print(f" -> 결과: {result}")
    time.sleep(1)

시도 1:
 -> 결과: 요청 실패: API 응답 실패
시도 2:
 -> 결과: 요청 실패: API 응답 실패
시도 3:
서킷 브레이커 작동! 10초 동안 차단됩니다.
 -> 결과: 요청 실패: API 응답 실패
시도 4:
 -> 결과: 서킷이 열려있습니다. 잠시 후 다시 시도하세요.
시도 5:
 -> 결과: 서킷이 열려있습니다. 잠시 후 다시 시도하세요.


## 4. 실전 예제

### 4.1 다층 폴백 시스템
여러 단계의 폴백을 구현하여 서비스 안정성을 높인다.

In [11]:
class MultiLevelFallback:
    """다층 폴백 시스템이다."""
    
    def __init__(self):
        self.client = instructor.patch(OpenAI())
    
    def get_response(self, prompt: str) -> str:
        """다층 폴백으로 응답을 생성한다."""
        
        try:
            # 1. 가장 성능 좋은 모델을 시도한다
            return self._primary_method(prompt)
        except Exception as e:
            print(f"1차 방법(gpt-4o-mini) 실패: {e}, 2차 방법 시도...")
            try:
                # 2. 비용 효율적인 모델을 시도한다
                return self._secondary_method(prompt)
            except Exception as e2:
                print(f"2차 방법(gpt-4o) 실패: {e2}, 3차 방법 시도...")
                try:
                    # 3. 미리 정의된 캐시를 사용한다
                    return self._tertiary_method(prompt)
                except Exception as e3:
                    # 4. 모든 방법이 실패하면 최종 폴백 메시지를 반환한다
                    return self._final_fallback()
    
    def _primary_method(self, prompt: str) -> str:
        """주요 방법: GPT-4o-mini를 사용한다."""
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    
    def _secondary_method(self, prompt: str) -> str:
        """대체 방법: GPT-4o를 사용한다."""
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    
    def _tertiary_method(self, prompt: str) -> str:
        """3차 방법: 캐시된 응답을 사용한다."""
        cache = {
            "안녕": "안녕하세요!",
            "도움": "무엇을 도와드릴까요?"
        }
        if prompt in cache:
            return cache[prompt]
        raise ValueError("캐시된 응답이 없습니다.")
    
    def _final_fallback(self) -> str:
        """최종 폴백: 기본 메시지를 반환한다."""
        return "현재 서비스 이용이 어렵습니다. 잠시 후 다시 시도해주세요."

# 사용 예시
fallback_system = MultiLevelFallback()
# 정상적인 경우, gpt-4o-mini가 응답한다 (API 키가 유효하다면)
result = fallback_system.get_response("AI에 대해 설명해주세요")
print(result)

AI(인공지능)는 인간의 지능을 기계가 모방하거나 구현할 수 있도록 하는 기술 및 연구 분야입니다. AI는 다양한 분야에 적용될 수 있으며, 주로 다음과 같은 영역에서 사용됩니다:

1. **기계 학습(Machine Learning)**: 데이터에서 패턴을 학습하고 이를 기반으로 예측이나 결정을 내리는 알고리즘입니다. 예를 들어, 이메일 스팸 필터링이나 추천 시스템에서 사용됩니다.

2. **자연어 처리(Natural Language Processing, NLP)**: 컴퓨터가 인간의 언어를 이해하고 생성할 수 있도록 하는 기술입니다. 예를 들어, 챗봇, 음성 인식 시스템, 번역 서비스에 활용됩니다.

3. **컴퓨터 비전(Computer Vision)**: 이미지나 비디오에서 정보를 추출하고 이해하는 기술입니다. 자율주행차, 얼굴 인식, 의료 영상 분석 등이 이에 해당합니다.

4. **로봇 공학(Robotics)**: AI를 활용하여 지능형 로봇을 개발하고 제어하는 분야입니다. 산업용 로봇, 드론, 개인용 로봇 등이 여기에 포함됩니다.

AI는 다양한 알고리즘과 모델, 데이터 구조를 활용하여 개발되며, 딥러닝(Deep Learning)과 같은 심층 신경망을 이용해 복잡한 문제를 해결하기도 합니다. AI는 이미 다양한 분야에서 혁신을 이끌고 있으며, 의료, 금융, 교통, 제조업 등 여러 산업에서 활용되고 있습니다. 

향후 AI의 발전은 더욱 가속화될 것으로 예상되며, 윤리적 문제와 사회적 영향에 대한 논의도 계속 이루어지고 있습니다.


### 4.2 타임아웃과 복구
타임아웃을 설정하고 실패 시 빠르게 복구한다.

In [12]:
from concurrent.futures import ThreadPoolExecutor, TimeoutError

class TimeoutRecovery:
    """타임아웃 복구 클래스다."""
    
    def __init__(self, timeout: int = 5):
        self.timeout = timeout
        self.client = OpenAI()
    
    def call_with_timeout(self, prompt: str) -> str:
        """타임아웃이 있는 호출을 수행한다."""
        
        with ThreadPoolExecutor(max_workers=1) as executor:
            future = executor.submit(self._make_call, prompt)
            
            try:
                result = future.result(timeout=self.timeout)
                return result
            except TimeoutError:
                return self._handle_timeout()
            except Exception as e:
                return f"호출 중 오류 발생: {e}"
    
    def _make_call(self, prompt: str) -> str:
        """실제 API 호출을 수행한다."""
        # 일부러 시간을 지연시켜 타임아웃을 유발할 수 있다
        # time.sleep(10)
        response = self.client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    
    def _handle_timeout(self) -> str:
        """타임아웃 발생 시 처리한다."""
        return "응답 시간이 초과되었습니다. 더 간단한 질문으로 다시 시도해보세요."

# 사용 예시
# 타임아웃을 1초로 짧게 설정하여 테스트한다
timeout_recovery = TimeoutRecovery(timeout=1)
result = timeout_recovery.call_with_timeout("매우 복잡하고 긴 보고서를 요약해줘")
print(result)

응답 시간이 초과되었습니다. 더 간단한 질문으로 다시 시도해보세요.


### 4.3 검증 기반 복구
응답을 검증하고 잘못된 경우 재생성한다. LLM을 사용하여 LLM의 출력을 검증하는 흥미로운 패턴이다.

In [13]:
class ResponseValidator(BaseModel):
    """응답 검증 모델이다."""
    is_valid: bool = Field(..., description="응답이 원래 질문에 적절한지 여부")
    reason: str = Field(..., description="유효성 판단에 대한 이유")
    corrected_response: Optional[str] = Field(None, description="응답이 유효하지 않을 경우, 수정된 버전의 응답")

class ValidationRecovery:
    """검증 기반 복구 시스템이다."""
    
    def __init__(self):
        self.client = instructor.patch(OpenAI())
    
    def get_validated_response(self, prompt: str, max_attempts: int = 3) -> str:
        """검증된 응답을 생성한다."""
        
        for attempt in range(max_attempts):
            response = self._generate_response(prompt)
            validation = self._validate_response(response, prompt)
            
            if validation.is_valid:
                return response
            
            print(f"응답 검증 실패 (시도 {attempt + 1}): {validation.reason}")
            
            if validation.corrected_response:
                print("수정된 응답을 사용합니다.")
                return validation.corrected_response
        
        return "최종 폴백: 유효한 응답을 생성할 수 없습니다."
    
    def _generate_response(self, prompt: str) -> str:
        """응답을 생성한다."""
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    
    def _validate_response(self, response: str, original_prompt: str) -> ResponseValidator:
        """응답을 검증한다."""
        validation_prompt = f"""
        원래 질문: {original_prompt}
        생성된 응답: {response}
        
        이 응답이 원래 질문에 적절하게 답하고 있는지 검증하고, 만약 부적절하다면 수정된 응답을 제공하라.
        """
        
        validation = self.client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": validation_prompt}],
            response_model=ResponseValidator,
        )
        
        return validation

# 사용 예시
validator = ValidationRecovery()
result = validator.get_validated_response("파이썬의 주요 특징은?")
print(result)

파이썬은 여러 가지 주요 특징을 가지고 있어 다양한 프로그래밍 용도로 널리 사용되고 있습니다. 주요 특징은 다음과 같습니다:

1. **간결하고 가독성 높은 문법**: 파이썬은 코드가 쉽게 읽히고 이해할 수 있도록 설계되었습니다. 이는 개발자가 코드를 작성하고 유지보수하는 데 큰 도움이 됩니다.

2. **인터프리터 언어**: 파이썬은 인터프리터 언어로, 코드를 한 줄씩 실행할 수 있어 디버깅이 용이합니다. 이는 빠른 프로토타입 개발에 유리합니다.

3. **다양한 라이브러리 지원**: 파이썬은 데이터 과학, 웹 개발, 인공지능, 머신러닝 등 다양한 분야에 적용할 수 있는 방대한 표준 라이브러리와 서드파티 라이브러리를 제공합니다.

4. **객체 지향 프로그래밍(OOP)**: 파이썬은 객체 지향 프로그래밍을 지원하여 코드의 재사용성과 모듈화를 쉽게 할 수 있습니다.

5. **크로스 플랫폼**: 윈도우, 리눅스, macOS 등 다양한 운영 체제에서 실행할 수 있어 호환성이 뛰어납니다.

6. **동적 타이핑**: 변수의 데이터 타입을 명시적으로 선언할 필요가 없고, 값을 할당할 때 자동으로 타입이 결정됩니다. 이는 코드 작성 시 유연성을 제공합니다.

7. **확장성**: C, C++ 등의 다른 언어로 작성된 코드를 쉽게 통합할 수 있어 고성능의 애플리케이션 개발이 가능합니다.

8. **대화형 프로그래밍**: 파이썬은 대화형 셸을 제공하여 즉시 코드를 실행하고 결과를 확인하면서 개발할 수 있습니다.

9. **커뮤니티와 지원**: 파이썬은 큰 개발자 커뮤니티와 풍부한 문서, 튜토리얼, 포럼 등을 가지고 있어 학습과 문제 해결이 용이합니다.

10. **컨벤션 중심의 프로그래밍**: PEP(파이썬 개선 제안서)와 같은 가이드라인을 통해 프로그래머들 간의 일관성을 유지할 수 있습니다.

이러한 특징 덕분에 파이썬은 초보자부터 전문가까지 폭넓게 사용되고 있는 인기 있는 프로그래밍 언어입니다.


## 5. 복구 전략 모범 사례

### 5.1 명확한 오류 메시지
사용자와 개발자 모두를 위한 명확한 메시지를 제공한다.

In [14]:
class ErrorHandler:
    """오류 처리 클래스다."""
    
    @staticmethod
    def handle_error(error: Exception, user_friendly: bool = True) -> str:
        """오류를 처리하고 적절한 메시지를 반환한다."""
        
        error_messages = {
            "RateLimitError": {
                "user": "현재 요청이 많아 잠시 대기가 필요합니다.",
                "dev": f"Rate limit exceeded: {str(error)}"
            },
            "APITimeoutError": {
                "user": "응답 시간이 초과되었습니다.",
                "dev": f"Timeout: {str(error)}"
            }
        }
        
        error_type = type(error).__name__
        messages = error_messages.get(error_type, {
            "user": "알 수 없는 오류가 발생했습니다.",
            "dev": str(error)
        })
        
        return messages["user"] if user_friendly else messages["dev"]

# 사용 예시
try:
    raise RateLimitError("Too many requests", response=None, body=None)
except Exception as e:
    print(f"사용자 메시지: {ErrorHandler.handle_error(e, user_friendly=True)}")
    print(f"개발자 메시지: {ErrorHandler.handle_error(e, user_friendly=False)}")

사용자 메시지: 알 수 없는 오류가 발생했습니다.
개발자 메시지: 'NoneType' object has no attribute 'request'


### 5.2 로깅과 모니터링
복구 과정을 기록하여 문제를 추적한다.

In [15]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class LoggedRecovery:
    """로깅이 포함된 복구 시스템이다."""
    
    def __init__(self):
        self.client = OpenAI()
    
    def call_with_logging(self, prompt: str, max_retries: int = 3) -> str:
        """로깅과 함께 호출을 수행한다."""
        
        for attempt in range(max_retries):
            try:
                logger.info(f"시도 {attempt + 1}: {prompt[:50]}...")
                
                response = self.client.chat.completions.create(
                    model=MODEL,
                    messages=[{"role": "user", "content": prompt}]
                )
                
                logger.info(f"성공 (시도 {attempt + 1})")
                return response.choices[0].message.content
                
            except Exception as e:
                wait_time = 2 ** attempt
                logger.error(f"실패 (시도 {attempt + 1}): {e}. {wait_time}초 후 재시도")
                time.sleep(wait_time)
        
        logger.critical(f"모든 재시도 실패: {prompt[:50]}...")
        return "요청을 완료할 수 없습니다."

# 사용 예시
logged_recovery = LoggedRecovery()
result = logged_recovery.call_with_logging("테스트 프롬프트")
print(result)

2025-10-06 08:45:00,940 - INFO - 시도 1: 테스트 프롬프트...
2025-10-06 08:45:01,929 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-06 08:45:01,946 - INFO - 성공 (시도 1)


안녕하세요! 어떤 도움이 필요하신가요? 궁금한 점이나 요청사항이 있다면 말씀해 주세요.


### 5.3 복구 메트릭 수집
복구 성공률과 패턴을 추적하여 시스템의 안정성을 평가한다.

In [16]:
from collections import defaultdict

class RecoveryMetrics:
    """복구 메트릭 수집 클래스다."""
    
    def __init__(self):
        self.metrics = defaultdict(int)
        self.total_attempts_on_failure = 0
    
    def record_call(self, success: bool, attempts: int):
        """호출 결과를 기록한다."""
        if success:
            self.metrics["success_count"] += 1
        else:
            self.metrics["failure_count"] += 1
            self.total_attempts_on_failure += attempts
    
    def get_stats(self) -> dict:
        """통계를 반환한다."""
        total = self.metrics["success_count"] + self.metrics["failure_count"]
        success_rate = self.metrics["success_count"] / total if total > 0 else 0
        
        return {
            "total_requests": total,
            "success_count": self.metrics["success_count"],
            "failure_count": self.metrics["failure_count"],
            "success_rate": f"{success_rate:.2%}",
        }

# 사용 예시
metrics = RecoveryMetrics()
metrics.record_call(success=True, attempts=1)
metrics.record_call(success=True, attempts=2)
metrics.record_call(success=False, attempts=3)
print(metrics.get_stats())

{'total_requests': 3, 'success_count': 2, 'failure_count': 1, 'success_rate': '66.67%'}
